In [1]:
import os
import io
import sys
import json
import time
import random
import logging

import numpy as np
import pandas as pd
import datetime

import plotly.io as pio
import streamlit as st
import seaborn as sns
import zipfile

import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error

from utils.manager.login import *
from utils.inputs.validation import *
from utils.inputs.preprocess import *
from utils.inputs.ads import *
from utils.modeling.skforecast_utils import *
from utils.modeling.sktime_utils import *

# Set up the logging configuration for cmdstanpy
logger = logging.getLogger()

# Add NullHandler with CRITICAL log level
null_handler = logging.NullHandler()
null_handler.setLevel(logging.CRITICAL)
logger.addHandler(null_handler)

# Add StreamHandler with INFO log level
stream_handler = logging.StreamHandler()
stream_handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
stream_handler.setLevel(logging.INFO)
logger.addHandler(stream_handler)

logger.propagate = False

## Inputs

In [2]:
# Add dropdown for Country
country_name = "CA"

# Add dropdown for frequency
forecast_freq = "D"

# Add dropdown for data selection
data_selection = False

# Add dropdown for data selection
external_features = False

# Add file uploader to the sidebar
uploaded_file = 'Agency Services.csv'

In [3]:
if forecast_freq == "D":
    forecast_period = 92
elif forecast_freq == "B":
    forecast_period = 66
elif forecast_freq == "W":
    forecast_period = 26
elif forecast_freq == "M":
    forecast_period = 12

In [4]:
# Organize these selections into a dictionary
run_params = {
    "country_name": country_name,
    "forecast_freq": forecast_freq,
    "forecast_period": forecast_period,
    "data_selection": data_selection,
    "external_features": external_features,
    "weekend_weight": 5,
    "holiday_weight": 10,
    "metric_key": "mspe"
}

## Validation

In [5]:
try:
    # Validate the input file
    df = validate_input_file(uploaded_file, external_features)
    logging.info(f"Train Data Size: {df.shape}")
except Exception as e:
    # Log this exception or handle it further up the call stack
    raise Exception(f"An error occurred while validating the file: {str(e)}")

In [6]:
df

,ds,y
0,2018-01-01,0.0
1,2018-01-02,367.0
2,2018-01-03,391.0
3,2018-01-04,431.0
4,2018-01-05,395.0
...,...,...
360,2018-12-27,280.0
361,2018-12-28,278.0
362,2018-12-29,0.0
363,2018-12-30,0.0


## Processing

In [7]:
try:
    # Process the input file
    processed_df, forecast_df = process_input_file(df)
    logging.info(f"Train Data Size: {processed_df.shape}")
    logging.info(f"Forecast Data Size: {forecast_df.shape}")
except Exception as e:
    # Log this exception or handle it further up the call stack
    raise Exception(f"An error occurred while processing the file: {str(e)}")

-1


## Automated Data Selection

In [8]:
try:
    if data_selection:
        
        # Find optimal window 
        optimal_window_size = find_optimal_window(processed_df)
            
        logging.info(f"Optimal Window Size: {optimal_window_size}")

        # Add 180 days for feature engineering to optimal window
        optimal_window_size += 180

    else:
        optimal_window_size = len(processed_df)
        
except Exception as e:
    # Log this exception or handle it further up the call stack
    raise Exception(f"An error occurred while finding the optimal window: {str(e)}")
    

run_params["optimal_window_size"] = optimal_window_size
    
# Truncate the train set based on optimal window
optimal_df = processed_df[-optimal_window_size:].copy(deep=True)
    
logging.info(f"Optimal Train Data Size: {optimal_df.shape}")

# Find the min data for optimal train data
run_params["optimal_window_date"] = optimal_df['ds'].min()

In [9]:
optimal_window_size

365

## Final Data Checks

In [10]:
# Set forecast start and end dates
min_forecast_date = optimal_df['ds'].min() + pd.Timedelta(days=1)
max_forecast_date = min_forecast_date + pd.Timedelta(days=forecast_period)
logging.info(f"Forecast Range: {min_forecast_date} to {max_forecast_date}")

try:
    # Validate column counts based on whether external features are used
    if external_features:
        assert optimal_df.shape[1] > 2 and forecast_df.shape[1] > 2
    else:
        assert optimal_df.shape[1] == 2 and forecast_df.shape[1] == 2
    # Ensure non-empty data structure
    assert optimal_df.shape[1] > 0
    # Ensure same number of columns
    assert optimal_df.shape[1] == forecast_df.shape[1]
except Exception as e:
    raise ValueError("Invalid input data format.")

try:
    # Check coverage of forecast period by data
    if external_features:
        assert forecast_df['ds'].max() > max_forecast_date
except Exception as e:
    raise Exception("Incomplete external variable coverage for forecast period.")

In [11]:
# Get the names of the exogenous variables from the train data
run_params["exog_cols"] = list((optimal_df.columns).difference(['y', 'ds']))

In [12]:
def resample_dataframe(df, forecast_freq='D'):
    """
    Resample and compute the mean for the dataframes based on a specified frequency.
    """
    
    df['ds'] = pd.to_datetime(df['ds'])
    df.set_index('ds', inplace=True)
    df = df.resample(forecast_freq).mean()
    
    return df.reset_index()

In [13]:
try:
    # Generate date features
    optimal_df = resample_dataframe(optimal_df, forecast_freq)
    forecast_df = resample_dataframe(forecast_df, forecast_freq)
except Exception as e:
    raise Exception(f"Failed to set the data frequency to {forecast_freq}: {e}")

In [14]:
import pandas as pd
import warnings
import holidays  # Ensure the holidays library is installed and imported

def generate_date_features(df: pd.DataFrame, freq='D', country_name=None) -> pd.DataFrame:
    """
    Add time-based features to a DataFrame based on its DateTime index, considering the frequency of data.
    """
    
    df['ds'] = pd.to_datetime(df['ds'])
    df.set_index('ds', inplace=True)
    
    if not isinstance(df.index, pd.DatetimeIndex):
        error_message = "DataFrame must have a DateTimeIndex"
        logger.error("DataFrame must have a DateTimeIndex")
        raise ValueError(error_message)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")  # Suppress warnings during feature generation

        # Generate features based on the specified frequency
        if freq in ['D', 'B']:
            # Features specific to daily data
            df['day_of_week'] = df.index.dayofweek + 1  # Monday=1, Sunday=7
            df['day_of_year'] = df.index.dayofyear
            df['is_weekend'] = df.index.dayofweek.isin([5, 6]).astype(int)

        if freq in ['D', 'B', 'W']:  # Weekly features include week_of_year
            df['week_of_year'] = df.index.isocalendar().week.astype(int)

        # Features applicable to all frequencies
        df['quarter'] = df.index.quarter
        df['month'] = df.index.month
        df['year'] = df.index.year

        # Calculate holidays if country_name is provided
        if country_name:
            country_holidays = holidays.CountryHoliday(country_name)
            if freq in ['D', 'B']:
                # Mark holidays for daily data
                df['is_holiday'] = df.index.map(lambda date: int(date in country_holidays))
            elif freq == 'W':
                # Count holidays in a week for weekly data
                df['is_holiday'] = df.index.map(lambda week_start: sum(
                    1 for day in pd.date_range(start=week_start - pd.Timedelta(days=6), end=week_start)
                    if day in country_holidays))
            elif freq == 'M':
                # Count holidays in a month for monthly data
                df['is_holiday'] = df.index.map(lambda month_start: sum(
                    1 for day in pd.date_range(start=month_start.replace(day=1), end=month_start)
                    if day in country_holidays))

    return df.reset_index()

In [15]:
try:
    # Generate date features
    optimal_df = generate_date_features(optimal_df, forecast_freq, country_name)
    forecast_df = generate_date_features(forecast_df, forecast_freq, country_name)
except Exception as e:
    raise ValueError(f"Failed to generate features using 'ds': {e}")

In [16]:
# Get the names of the exogenous variables from the train data
run_params["exog_cols_all"]  = list((optimal_df.columns).difference(['y', 'ds']))

In [17]:
def determine_params(forecast_freq):
    """
    Determines lag window and test step size based on the frequency and weekend inclusion.
    """
    # Define settings for different scenarios using dictionaries
    # initial_window_size, lag_window_range, rolling_window_range, test_size, test_steps
    freq_settings = {
        "D": (90, [7, 15, 30, 60, 90], [3, 7, 15, 30, 60, 90], 30, 3),
        "B": (60, [5, 10, 20, 40, 60], [3, 5, 10, 20, 40, 60], 20, 2),
        "W": (12, [4, 8, 12], [4, 8, 12, 16, 20, 24], 6, 1),
        "M": (3, [3], [3, 6, 9, 12], 3, 1)
    }
    
    try:
        # Select the appropriate settings based on forecast frequency and weekend drop
        if forecast_freq in freq_settings:
            if isinstance(freq_settings[forecast_freq], dict):
                # Handle daily frequency differently based on weekend inclusion
                return freq_settings[forecast_freq][weekend_drop]
            else:
                return freq_settings[forecast_freq]
        else:
            raise ValueError(f"Unknown Frequency: {forecast_freq}")
    except Exception as e:
        error_message = f"Failed to determine lag window and test set size: {e}"
        logger.error(error_message)
        raise Exception(error_message)

In [18]:
try:
    initial_window_size, lag_window_range, rolling_window_range, test_size, test_steps = determine_params(forecast_freq)
    logger.info(f"Initial Window Size: {initial_window_size}, Lag Window Range: {lag_window_range}")
    logger.info(f"Test Size: {test_size}, Test Steps: {test_steps}")
except Exception as e:
    raise Exception(e)

In [19]:
run_params.update({
    "initial_window_size": initial_window_size,
    "lag_window_range": lag_window_range,
    "rolling_window_range": rolling_window_range,
    "test_size": test_size,
    "test_steps": test_steps
})

In [20]:
try:
    test_df = optimal_df[-test_size:].copy(deep=True)
    test_df = test_df.set_index('ds').asfreq(forecast_freq)
    
    train_df = optimal_df[:-test_size].copy(deep=True)
    train_df = train_df.set_index('ds').asfreq(forecast_freq)
    assert len(train_df) + len(test_df) == len(optimal_df)
except Exception as e:
    raise ValueError(f"Failed to split into train and test: {e}")

In [21]:
if 'is_holiday' in train_df:
    train_holiday_mask = train_df['is_holiday'].values
if 'is_holiday' in test_df:
    test_holiday_mask = test_df['is_holiday'].values

In [22]:
run_params

{'country_name': 'CA',
 'forecast_freq': 'D',
 'forecast_period': 92,
 'data_selection': False,
 'external_features': False,
 'weekend_weight': 5,
 'holiday_weight': 10,
 'metric_key': 'mspe',
 'optimal_window_size': 365,
 'optimal_window_date': Timestamp('2018-01-01 00:00:00'),
 'exog_cols': [],
 'exog_cols_all': ['day_of_week',
  'day_of_year',
  'is_holiday',
  'is_weekend',
  'month',
  'quarter',
  'week_of_year',
  'year'],
 'initial_window_size': 90,
 'lag_window_range': [7, 15, 30, 60, 90],
 'rolling_window_range': [3, 7, 15, 30, 60, 90],
 'test_size': 30,
 'test_steps': 3}

In [23]:
import os
import yaml

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sktime.forecasting.fbprophet import Prophet
from sktime.forecasting.naive import NaiveForecaster

def load_model_params_and_create_instance(model_type, current_dir):
    """
    Load model parameters from a YAML file and create a model instance based on model type.
    """
    # Dictionary to map model types to their respective classes and YAML files
    model_config = {
        'random_forest': (RandomForestRegressor(), 'random_forest.yaml'),
        'xgboost': (XGBRegressor(), 'xgboost.yaml'),
        'prophet': (Prophet(), 'prophet.yaml'),
        'naive': (NaiveForecaster(), 'naive.yaml')
    }
    
    # Ensure the model type is supported
    if model_type not in model_config:
        raise ValueError(f"Unsupported model type: {model_type}")
    
    model_class, yaml_file = model_config[model_type]
    file_path = os.path.join(current_dir, 'params', yaml_file)
    
    # Load parameters from the YAML file, handling file and parsing errors
    try:
        with open(file_path, 'r') as file:
            param_grid = yaml.safe_load(file)
    except FileNotFoundError:
        raise FileNotFoundError(f"The configuration file {yaml_file} was not found in {file_path}")
    except yaml.YAMLError as e:
        raise Exception(f"Error parsing the YAML file: {e}")

    return model_class, param_grid

In [24]:
current_dir = 'utils/modeling'

In [25]:
search_results = {}

for model_type in ['random_forest', 'xgboost']:
    # Load parameters for grid search
    model, param_grid = load_model_params_and_create_instance(model_type, current_dir)
    
    # Find best model
    best_configuration, all_results, best_model = find_best_model_skforecast(
        lag_window_range, model, train_df, param_grid, run_params
    )
    
    # Save best model and config
    search_results[model_type] = {
        'best_model': best_model,
        'best_configuration': best_configuration,
        'all_results': all_results,
        'package_type': 'skforecast'
    }
    
    
for model_type in ['prophet', 'naive']:
    # Load parameters for grid search
    model, param_grid = load_model_params_and_create_instance(model_type, current_dir)
    
    # Find best model
    best_configuration, all_results, best_model = find_best_model_sktime(
        train_df['y'], run_params, model, param_grid
    )
    
    # Save best model and config
    search_results[model_type] = {
        'best_model': best_model,
        'best_configuration': best_configuration,
        'all_results': all_results,
        'package_type': 'sktime'
    }

Number of models compared: 1.


lags grid:   0%|          | 0/1 [00:00<?, ?it/s]

params grid:   0%|          | 0/1 [00:00<?, ?it/s]

Number of models compared: 1.


lags grid:   0%|          | 0/1 [00:00<?, ?it/s]

...

lags grid:   0%|          | 0/1 [00:00<?, ?it/s]

params grid:   0%|          | 0/1 [00:00<?, ?it/s]

Number of models compared: 1.


lags grid:   0%|          | 0/1 [00:00<?, ?it/s]

params grid:   0%|          | 0/1 [00:00<?, ?it/s]